## Main Authors:
* Fabbrico Davide
* Khateeb Ahmad Arsalan
* Pagnoncelli Diego

In [52]:
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import datetime
import plotly.express as px
import plotly.io as pio

pd.set_option('display.max_rows', 1000)
pd.set_option('mode.chained_assignment', None)
pio.templates.default = "ggplot2"

In [53]:
# read total dataframe
df_tot = pd.read_csv('total_dataframe.csv').drop('Unnamed: 0', axis=1)

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning:

Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.



In [54]:
def PlotGreenGraph(G):
    labels = {e: str(e) for e in G.edges}
    # pos = nx.spring_layout(G, k=10000/np.sqrt(G.order()), iterations=20, scale=10) problem divided by 0
    pos = nx.spring_layout(G, iterations=20, scale=10)
    plt.figure(3, figsize=(20, 14))
    
    nx.draw(G, pos=pos, with_labels=True, node_size=2000, node_color='orange')

    # Create edge labels
    labels = {e: G.edges[e]['Quantity'] for e in G.edges}

    # Draw edge labels according to node positions
    nx.draw_networkx_edge_labels(G, pos, edge_labels=labels)
    
    return None

In [55]:
def GreennessGraph(df, month = 1, year = 2019, type_flow = 'export', product = None, t = 1):
    
    products = df_tot.PRODUCT_CODE.unique()
    products = [p for p in products if product in str(p)[:2]]
    
    # product
    if product != None or product != '':
        df = df[df['PRODUCT_CODE'].isin(products)].reset_index(drop=True)
    else:
        return 'Select the product code!'
    
    # month and year
    if month == None or year == None or month > 12 or year > 2021 or year < 2019:
        return 'Insert correctly the month and year!'
    
    if month != None and year != None:
        df = df[(df['MONTH'] == month) & (df['YEAR'] == year)].reset_index(drop=True)

    df = df[df['TYPE_FLOW'] == type_flow].reset_index(drop=True) 
    
    df['Distance'] = round(df['Distance'], 3)
    
    # start changes
    df = df.groupby(['DECLARANT_ISO', 'PARTNER_ISO', 'PRODUCT_CODE','TYPE_TRANSP_MODE', 'Distance'])[['DECLARANT_ISO', 'PARTNER_ISO', 'VALUE_IN_EUROS', 'QUANTITY_IN_KG']].sum().reset_index()
    df = df.loc[df['TYPE_TRANSP_MODE'].isin(['Road', 'Air', 'Sea', 'Rail','Inland Waterway'])].reset_index(drop=True)
    speed = 45
    
    def greeness(row):
        if row['TYPE_TRANSP_MODE'] == 'Sea':
            return (row['QUANTITY_IN_KG']*(row['Distance']/1000)*9936.70)/((24000*7000)*speed)
        elif row['TYPE_TRANSP_MODE'] == 'Air':
            return (row['QUANTITY_IN_KG']*12*(row['Distance']/1000))/77000
        # elif row['TYPE_TRANSP_MODE'] == 'Inland Waterway':
            # return (row['QUANTITY_IN_KG']*0.12*(row['Distance']/1000)) # find the maximum capacity
        elif row['TYPE_TRANSP_MODE'] == 'Road':
            return (row['QUANTITY_IN_KG']*(row['Distance']/1000)*0.25)/(30000)
        elif row['TYPE_TRANSP_MODE'] == 'Rail':
            return (row['QUANTITY_IN_KG']*(row['Distance']/1000)*7.97)/(528000)

    df['Greenness'] = df.apply(lambda row: greeness(row), axis=1) 
    
    tr = int(len(df)*t/100)
    df = df.iloc[:tr+1]
    
    G = nx.DiGraph()
    # nodes
    nodes = df['DECLARANT_ISO'].append(df['PARTNER_ISO']).unique()
    
    # graph
    G.add_nodes_from(nodes)

    for index,row in df.iterrows():
        G.add_edge(row['DECLARANT_ISO'],row['PARTNER_ISO'],
                                             Value = row.VALUE_IN_EUROS,
                                             Quantity = row.QUANTITY_IN_KG,
                                            Distance = row.Distance,
                                            Greenness = row.Greenness)

    
    return G

In [56]:
def DFGreenGraph(G, month=None, year=None):
    dict_summ = {}
    for i in G.nodes():
        summ = 0
        for j in G[i]:
            summ += G[i][j]['Greenness'] # for each node I calculate the sum of the GREENNESS of the out-edge
        dict_summ[i] = summ
        
    dict_summQ = {}
    for i in G.nodes():
        summQ = 0
        for j in G[i]:
            summQ += G[i][j]['Quantity'] # for each node I calculate the sum of the GREENNESS of the out-edge
        dict_summQ[i] = summQ

    df_greenG = pd.DataFrame(list(dict_summ.items()), columns=['Country', 'Greenness']).sort_values(by='Greenness', ascending=False).reset_index(drop=True)
    df_greenQ = pd.DataFrame(list(dict_summQ.items()), columns=['Country', 'Quantity'])
    df_GQ = df_greenG.merge(df_greenQ, on='Country')
    df_GQ['month'] = month
    df_GQ['year'] = year
    df_GQ['date'] = datetime.datetime(year, month, 1)
    df_GQ = df_GQ[df_GQ['Greenness']>0]
    return df_GQ

In [57]:
year = [2019, 2020, 2021]
month = [1,2,3,4,5,6,7,8,9,10,11,12]
df = pd.DataFrame({})
for y in year:
    print(str(y))
    if y == 2021:
        month = list(range(1, 7))   
    for m in month:
        print(str(m))
        greenG = GreennessGraph(df_tot, m, y, product = '72', t = 10)
        greenDF = DFGreenGraph(greenG, month=m, year=y)
        df = pd.concat([df, greenDF], copy=False)
df = df.reset_index(drop=True)

2019
1
2
3
4
5
6
7
8
9
10
11
12
2020
1
2
3
4
5
6
7
8
9
10
11
12
2021
1
2
3
4
5
6


In [160]:
df1 = df.copy()

In [161]:
df_country = pd.DataFrame({})
for i in df['Country'].unique():
    df_country[i] = df1[df1['Country'] == i]['Greenness'].to_list()
    
for i in df['Country'].unique():
    df_country['year'] = df1[df1['Country'] == i]['year'].to_list()
    df_country['month'] = df1[df1['Country'] == i]['month'].to_list()

## Multiple Line

In [162]:
hmc = len(df1['Country'].unique())
fig = px.line(df_country, x="month", y=df_country.columns[:hmc], animation_frame="year", markers=True)
fig["layout"]
fig.layout.updatemenus[0].buttons[0].args[1]["frame"]["duration"] = 1500
fig.update_layout(
    title="Index of Greenness (2019-2021)",
    xaxis_title="Month",
    yaxis_title="Greenness",
    title_x=0.5,
    font=dict(
        size=16,
    ),
    yaxis_range=[min(df1['Greenness']-3000),max(df1['Greenness']+3000)],
    xaxis_range = [0.5,12.5]
)
# fig.update_layout(title_text='Index of Greenness (2019-2021)', title_x=0.5)
fig.update_xaxes(tickmode = 'array',
                 tickvals = sorted(df1['month'].unique()),
                 ticktext= ['Jan', 'Feb', 'Mar', 'Apr', 'May','Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])
fig.write_html("/Users/davide/Desktop/moreline.html")

## One Line

In [163]:
df2 = df1[df1['Country']=='BE']
fig = px.line(df2, x="month", y="Greenness", animation_frame="year", animation_group="Country", hover_name="Country", markers=True)
fig["layout"]
fig.layout.updatemenus[0].buttons[0].args[1]["frame"]["duration"] = 1500
fig.update_layout(
    title="Index of Greenness (2019-2021)",
    xaxis_title="Month",
    yaxis_title="Greenness",
    title_x=0.5,
    font=dict(
        size=16,
    ),
    yaxis_range=[min(df2['Greenness']-3000),max(df2['Greenness']+111000)],
    xaxis_range = [0.5,12.5]
)
# fig.update_layout(title_text='Index of Greenness (2019-2021)', title_x=0.5)
fig.update_xaxes(tickmode = 'array',
                 tickvals = sorted(df2['month'].unique()),
                 ticktext= ['Jan', 'Feb', 'Mar', 'Apr', 'May','Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])
fig.write_html("/Users/davide/Desktop/oneline.html")

In [164]:
ccode_df = pd.read_csv('country_codes.csv')
map_dict1 = pd.Series(ccode_df.iloc[:,0].values,index= ccode_df.iloc[:,1]).to_dict()
map_dict2 = pd.Series(ccode_df.iloc[:,2].values,index= ccode_df.iloc[:,0]).to_dict()
df1.replace({"Country":map_dict1},inplace=True)
df1['Code'] = df1['Country'].map(map_dict2)
df1['Date'] = df1['date'].dt.strftime("%b-%Y").astype(str)

In [165]:
fig = px.choropleth(df1, locations="Code",
                    scope = "europe",
                    color="Greenness", # lifeExp is a column of gapminder
                    hover_name="Country", # column to add to hover information
                    color_continuous_scale="Greens",
                    animation_frame="Date"
                    )
fig.update_layout(margin={"r":20,"t":20,"l":2,"b":200})
fig.show()
fig.write_html("/Users/davide/Desktop/map.html")
#fig.write_html("C:\\Users\\Arsalan Khateeb\\Coding_Lab\\sta-py\\file.html")